# Table 7.6

Theme Song: Smoke on the Water<br>
Artist: Deep Purple<br>
Album: Machine Head<br>
Released: 1972

- Add 5% to fixed-SST ERF for surface cooling

In [1]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as pl
import pandas as pd
from matplotlib import gridspec, rc
from matplotlib.lines import Line2D
from netCDF4 import Dataset
import warnings

from ar6.constants import NINETY_TO_ONESIGMA

In [2]:
# # # add models from Zelinka 14
z14_data = np.array([
    [-0.66, 0.28, -0.24, -0.01, 0.15, 0.01, -0.22], #'IPSL-CM5A-LR'
    [-0.58, 0.17, -0.51, 0.01, -0.01, 0.11, -0.04], #'CanESM2'
    [-0.63, 0.29, -0.80, 0.03, 0.18, 0.13, -0.17], #'NorESM1-M'
    [-1.13, 0.49, -0.76, 0.08, 0.00, -0.02, -0.21], #'CSIRO-Mk3-6-0'
    [-0.53, 0.26, -1.00, 0.06, -0.13, 0.14, -0.05], #'HadGEM2-A'
    [-0.91, 0.41, -0.99, -0.03, -0.13, 0.11, 0.02], #'GFDL-CM3'
    [-0.66, 0.16, -0.93, -0.01, -0.28, 0.22, 0.27], #'MIROC5'
    [-0.11, 0.12, -1.77, -0.09, -0.23, 0.00, 0.95], #'MRI-CGCM3'
    [-0.33, 0.21, -1.93, 0.06, -0.11, 0.17, 0.57], #'CESM1-CAM5'
])
print(z14_data[:,0])
ERFariCMIP5 = z14_data[:,0]+z14_data[:,1]+z14_data[:,5]
ERFaciCMIP5 = z14_data[:,2]+z14_data[:,3]+z14_data[:,4]+z14_data[:,6]
ERFCMIP5 = ERFariCMIP5+ERFaciCMIP5
print(ERFCMIP5)

[-0.66 -0.58 -0.63 -1.13 -0.53 -0.91 -0.66 -0.11 -0.33]
[-0.69 -0.85 -0.97 -1.55 -1.25 -1.52 -1.23 -1.13 -1.36]


In [3]:
df = pd.read_csv('../data_input/Smith_et_al_ACP_2020/aprp_aer_20200211.csv')
# treat GISS p1 and GISS p3 as separate models
df.loc[(df['Model']=='GISS-E2-1-G') & (df['Run']=='r1i1p1f1'),'Model'] = 'GISS-E2-1-G p1'
df.loc[(df['Model']=='GISS-E2-1-G') & (df['Run']=='r1i1p1f1'),'Model'] = 'GISS-E2-1-G p3'
df

,Model,Run,ERF,ERF_SW,ERF_LW,ERFariSW_sca,ERFariSW_abs,ERFaciSW_sca,ERFaciSW_abs,ERFaciSW_clt,...,ERFari,ERFaci,ERFariaci,od550aer,ghanERFariSW,ghanERFaciSW,ghanERFariLW,ghanERFaciLW,ghanERFari,ghanERFaci
0,ACCESS-CM2,r1i1p1f1,-1.093110,-1.395235,0.302125,-0.785361,0.300986,-0.798324,-0.008777,-0.124594,...,-0.224056,-0.889889,-1.113945,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ACCESS-ESM1-5,r1i1p1f1,-1.143741,-1.289711,0.145970,-0.428137,0.196440,-1.014394,0.038882,-0.131532,...,-0.063370,-1.129400,-1.192771,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BCC-ESM1,r1i1p1f1,-1.473334,-1.694332,0.220998,-1.124120,0.215463,-0.575091,-0.065881,-0.084571,...,-0.752415,-0.660787,-1.413203,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CanESM5,r1i1p2f1,-0.851293,-0.832269,-0.019024,-0.599152,0.518832,-0.883347,0.057386,-0.136031,...,-0.020971,-1.040365,-1.061337,NaN,0.326426,-1.224674,0.035551,-0.048576,0.361977,-1.273250
4,CESM2,r1i1p1f1,-1.371578,-1.527308,0.155730,-0.257211,0.351071,-1.701553,0.030856,-0.006556,...,0.146324,-1.573986,-1.427662,0.012937,NaN,NaN,NaN,NaN,NaN,NaN
5,CNRM-CM6-1,r1i1p1f2,-1.154830,-1.247711,0.092881,-0.611040,0.202989,-0.768677,-0.052504,0.054585,...,-0.262541,-0.819226,-1.081766,0.021007,-0.424447,-0.739004,0.002796,-0.051302,-0.421651,-0.790306
6,CNRM-ESM2-1,r1i1p1f2,-0.740564,-0.821450,0.080886,-0.420806,0.179076,-0.517683,-0.042614,-0.027340,...,-0.141798,-0.606682,-0.748481,0.016143,-0.217646,-0.590575,0.004485,-0.016657,-0.213161,-0.607232
7,EC-Earth3,r1i1p1f1,-0.798890,-0.841935,0.043045,-0.749645,0.317640,-0.344111,-0.037547,-0.079197,...,-0.370462,-0.479353,-0.849815,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,GFDL-CM4,r1i1p1f1,-0.731706,-0.798641,0.066934,-0.647860,0.409477,-0.539351,0.000186,-0.085712,...,-0.112430,-0.683897,-0.796327,0.039799,0.057721,-0.763368,0.017284,-0.044227,0.075006,-0.807595
9,GFDL-ESM4,r1i1p1f1,-0.697921,-0.588490,-0.109431,-0.647766,0.532049,-0.587167,0.008705,-0.055594,...,-0.055215,-0.803989,-0.859204,0.040951,0.235149,-0.770094,0.020959,-0.153508,0.256108,-0.923602


In [4]:
data = []
erfari = []
erfaci = []
total = []

for model in df.Model.unique():
    erfari.append(1.05 * df.ERFari[df.Model==model].mean())
    erfaci.append(1.05 * df.ERFaci[df.Model==model].mean())
    total.append(1.05 * df.ERFari[df.Model==model].mean() + 1.05 * df.ERFaci[df.Model==model].mean())
    data.append([model, erfari[-1], erfaci[-1], total[-1]])
erfari = np.array(erfari)
erfaci = np.array(erfaci)
total = np.array(total)
data.append(['CMIP6 mean', np.nanmean(erfari), np.nanmean(erfaci), np.nanmean(total)])
data.append(
    [
        'CMIP6 range',
        NINETY_TO_ONESIGMA * np.nanstd(erfari, ddof=1), 
        NINETY_TO_ONESIGMA * np.nanstd(erfaci, ddof=1),
        NINETY_TO_ONESIGMA * np.nanstd(total, ddof=1)
    ]
) # use sample stdev and inflate to 5-95%
data.append(['CMIP5 mean', np.mean(1.05 * ERFariCMIP5), np.mean(1.05 * ERFaciCMIP5), np.mean(1.05 * ERFCMIP5)])
data.append(
    [
        'CMIP5 range',
        NINETY_TO_ONESIGMA * np.nanstd(1.05 * ERFariCMIP5),
        NINETY_TO_ONESIGMA * np.nanstd(1.05 * ERFaciCMIP5),
        NINETY_TO_ONESIGMA * np.nanstd(1.05 * ERFCMIP5)
    ]
)

In [5]:
df = pd.DataFrame(data, columns = ['Model', 'ERFari', 'ERFaci', 'ERFari+aci'])

In [6]:
df

,Model,ERFari,ERFaci,ERFari+aci
0,ACCESS-CM2,-0.235259,-0.934384,-1.169643
1,ACCESS-ESM1-5,-0.066539,-1.185870,-1.252409
2,BCC-ESM1,-0.790036,-0.693827,-1.483863
3,CanESM5,-0.022020,-1.092384,-1.114403
4,CESM2,0.153640,-1.652685,-1.499045
5,CNRM-CM6-1,-0.275668,-0.860187,-1.135855
6,CNRM-ESM2-1,-0.148888,-0.637017,-0.785905
7,EC-Earth3,-0.388985,-0.503320,-0.892306
8,GFDL-CM4,-0.118052,-0.718092,-0.836143
9,GFDL-ESM4,-0.057976,-0.844188,-0.902164


In [7]:
df.to_csv('../data_output/table7.6.csv', index=False)